In [1]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, accuracy_score, log_loss, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import FeatureAgglomeration
import category_encoders as ce
from numpy.testing import assert_almost_equal
from functools import reduce
import matplotlib.pyplot as plt
import time
from category_encoders.binary import BinaryEncoder
pd.set_option('display.max_columns', None)  # Unlimited columns
import xgboost as xgb
# from xgboost import XGBClassifier, XGBRegressor
import copy
from sklearn.feature_extraction.text import TfidfVectorizer
import kaggle
import re

In [2]:
np.__version__

'1.15.4'

In [3]:
xgb.__version__

'0.81'

In [4]:
url = "unit-3/train_features.csv"
df = pd.read_csv(url, parse_dates=['date_recorded'],index_col='id' )

In [5]:
df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
df.shape

(59400, 39)

In [7]:
df['timestamp'] = df.date_recorded.apply(lambda d: d.timestamp()/ 10 ** 9)
df_ = df.drop(['date_recorded', 'region_code', 'district_code', 'region'], inplace = False, axis=1)
#df_ = df.drop(['date_recorded', 'district_code', 'region'], inplace = False, axis=1)
df_['region_district'] = df.apply(lambda row: f'{row.region}_{row.district_code}' , axis=1)
# df_ = df_.apply(lambda x: x.astype(str).str.lower())
train_input_columns = list(df_.columns)
train_numeric_columns = df_.select_dtypes(exclude=['object']).columns
df_.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,timestamp,region_district
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1.300061,Iringa_5
8776,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.362528,Mara_2
34310,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1.361750,Manyara_4
67743,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1.359331,Mtwara_63
19728,0.0,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.310515,Kagera_1


In [8]:
df_.shape

(59400, 37)

In [9]:
dc = 'status_group'

In [10]:
yurl = 'unit-3/train_labels.csv'
dfy = pd.read_csv(yurl, index_col='id' )
dfy.shape
y = dfy[dc]

In [11]:
y.shape

(59400,)

In [12]:
oc = df_.select_dtypes(include=['object']).columns
oc

Index(['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'lga', 'ward',
       'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name',
       'permit', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'region_district'],
      dtype='object')

In [13]:
hot = []
binary = []
for o in oc:
    if df_[o].unique().shape[0] > 127:
        print(df_[o].unique().shape[0], o)
        binary.append(o)
    else:
        hot.append(o)

1898 funder
2146 installer
37400 wpt_name
19288 subvillage
2092 ward
2697 scheme_name
132 region_district


In [14]:
test_url = "unit-3/test_features.csv"
df = pd.read_csv(test_url, parse_dates=['date_recorded'],index_col='id' )
df['timestamp'] = df.date_recorded.apply(lambda d: d.timestamp()/ 10 ** 9)
dft = df.drop(['date_recorded', 'region_code', 'district_code', 'region'], inplace = False, axis=1)
# dft = df.drop(['date_recorded', 'district_code', 'region'], inplace = False, axis=1)
dft['region_district'] = df.apply(lambda row: f'{row.region}_{row.district_code}' , axis=1)
# dft = dft.apply(lambda x: x.astype(str).str.lower())
test_input_columns = list(dft.columns)
# dft[dc] = ['fuctional'] * dft.shape[0]
dft.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,timestamp,region_district
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,1.359936,Manyara_3
51630,0.0,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,1.359936,Arusha_2
17168,0.0,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other,1.359677,Singida_2
45559,0.0,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other,1.358813,Lindi_43
49871,500.0,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1.364342,Ruvuma_3


In [15]:
def regex_filter(val, regex):
    #print(val,regex)
    if str(type(val)) == "<class 'str'>":
        mo = re.search(regex,val)
        if mo:
            return True
        else:
            return False
    else:
        return False
def equal_filter(val, match):
    if str(type(val)) == "<class 'str'>":
        return val == match
    else:
        return False    

In [16]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [17]:
dft[dft.funder.apply(lambda f : regex_filter(f,  "^G[o,v]"))].funder.unique()
dft[dft.funder.apply(lambda f : regex_filter(f,  "^T[a,z]"))].funder.unique()

array(['Government Of Tanzania', 'Government/ World Bank', 'Go',
       'Government/ Community', 'Government /tassaf', 'Goldfinder',
       'Government/tassaf', 'Government/julius'], dtype=object)

array(['Tasaf', 'Tz Japan', 'Tanza', 'Tardo',
       'Tanzania And Egypt Cooperat', 'Tassaf', 'Tanap', 'Tanapa',
       'Tabora Municipal Council', 'Tado', 'Tacare', 'Tadepa', 'Ta',
       'Tanzakesho', 'Tassaf/ Danida', 'Tassaf Ii', 'Tag Church Vi',
       'Tambalizeni', 'Tassaf I', 'Tasaf/dmdd', 'Tanroad', 'Tasafu',
       'Tahea', 'Tasaf/village Community', 'Tackle', 'Tag',
       'Tanzania/australia', 'Tanzania Na Egypt', 'Tanzania',
       'Tanzania Nea Egypt', 'Tanzania Christian Service', 'Tasmin',
       'Tz/japan Embass'], dtype=object)

In [18]:
df_[df_.funder.apply(lambda f : regex_filter(f, "^G[o,v]"))].funder.unique()
df_[df_.funder.apply(lambda f : regex_filter(f,  "^T[a,z]"))].funder.unique()

array(['Government Of Tanzania', 'Go', 'Government/ Community',
       'Government /tassaf', 'Government/ World Bank', 'Government',
       'Government /sda', 'Godii', 'Government And Community',
       'Government Of Misri', 'Goldwill Foundation', 'Government/tassaf',
       'Government/school', 'Government/tcrs', 'Government /world Vision',
       'Goldmain'], dtype=object)

array(['Tasaf', 'Tanza', 'Tassaf', 'Tardo', 'Ta', 'Tado', 'Tanzania',
       'Tabora Municipal Council', 'Tasaf And Lga', 'Tasaf Ii', 'Tanesco',
       'Tz Japan', 'Tasae', 'Taboma', 'Tacare', 'Tassaf I', 'Tasafu',
       'Tag', 'Tanapa', 'Tanzakesho', 'Tasaf/dmdd', 'Tahea',
       'Tanz Egypt Technical Cooper', 'Tanroad', 'Taasaf', 'Tabraki',
       'Tanap', 'Tanzania Compasion', 'Tag Church Ub', 'Tacri',
       'Tag Church', 'Taes', 'Tassaf Ii', 'Tansi',
       'Tanzania Egypt Technical Co Op', 'Tasf', 'Tareto', 'Tasad',
       'Tanzaling', 'Tassaf/ Danida', 'Tajiri Jumbe Lila',
       'Tasaf And Mmem', 'Tasaf 1', 'Tasef', "Tag Patmo's",
       'Tanz/egypt Technical  Co-op', 'Tanedaps Society',
       'Tanzania Journey', 'Tadepa', 'Tancro', 'Taipo', 'Tabea', 'Tadeo',
       'Tanzania /egypt', 'Taees', 'Tancan', 'Tasaf/tlc',
       'Tarangire Park', 'Tanga Cement', 'Tanload', 'Tasa', 'Tz As'],
      dtype=object)

In [19]:
dft[dft.funder.apply(lambda f : regex_filter(f,  "julius"))][0:5]

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,timestamp,region_district
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43366,0.0,Government/julius,1499,Government,37.960115,-4.443879,Kwa Julius,0,Pangani,Msasa,Same,Kirangare,50,True,GeoData Consultants Ltd,Water authority,Kilotweni water supply,True,2010,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,1.363738,Kilimanjaro_3


In [20]:
replacements = [
    ('Go', 'Government Of Tanzania'),
    ('Government', 'Government Of Tanzania'),
    ('Government/tassaf', 'Government /tassaf'),
    ('Government And Communty', 'Government/ community'),
    ('Tanzania', 'Government Of Tanzania'),
    ('Government/julius', 'Government Of Tanzania')
]

for r in replacements:
    df_.loc[lambda d: d['funder'].apply(lambda f : equal_filter(f, r[0])), ['funder']] = r[1]
    dft.loc[lambda d: d['funder'].apply(lambda f : equal_filter(f, r[0])), ['funder']] = r[1]

            

In [21]:
df_[df_.funder.apply(lambda f : regex_filter(f, "^G[o,v]"))].funder.unique()

array(['Government Of Tanzania', 'Government/ Community',
       'Government /tassaf', 'Government/ World Bank', 'Government /sda',
       'Godii', 'Government And Community', 'Government Of Misri',
       'Goldwill Foundation', 'Government/school', 'Government/tcrs',
       'Government /world Vision', 'Goldmain'], dtype=object)

In [22]:
dft[dft.funder.apply(lambda f : regex_filter(f,  "^G[o,v]"))].funder.unique()
dft[dft.funder.apply(lambda f : regex_filter(f,  "^T[a,z]"))].funder.unique()

array(['Government Of Tanzania', 'Government/ World Bank',
       'Government/ Community', 'Government /tassaf', 'Goldfinder'],
      dtype=object)

array(['Tasaf', 'Tz Japan', 'Tanza', 'Tardo',
       'Tanzania And Egypt Cooperat', 'Tassaf', 'Tanap', 'Tanapa',
       'Tabora Municipal Council', 'Tado', 'Tacare', 'Tadepa', 'Ta',
       'Tanzakesho', 'Tassaf/ Danida', 'Tassaf Ii', 'Tag Church Vi',
       'Tambalizeni', 'Tassaf I', 'Tasaf/dmdd', 'Tanroad', 'Tasafu',
       'Tahea', 'Tasaf/village Community', 'Tackle', 'Tag',
       'Tanzania/australia', 'Tanzania Na Egypt', 'Tanzania Nea Egypt',
       'Tanzania Christian Service', 'Tasmin', 'Tz/japan Embass'],
      dtype=object)

In [23]:
dft.wpt_name.isna().sum()
df_.wpt_name.isna().sum()
dft.wpt_name.unique()

0

0

array(['Dinamu Secondary School', 'Kimnyak', 'Puma Secondary', ...,
       'Kwambwezi', 'Bonde La Mkondoa', 'Kwa Mzee Chagala'], dtype=object)

In [24]:
# tfidf_vectorizer = TfidfVectorizer(
#     min_df=1,  # min count for relevant vocabulary
#     strip_accents='unicode',  # replace all accented unicode char 
#     # by their corresponding  ASCII char
#     analyzer='word',  # features made of words
#     use_idf=True,  # enable inverse-document-frequency reweighting
#     smooth_idf=True,  # prevents zero division for unseen words
#     sublinear_tf=False)

# tfidf_df = tfidf_vectorizer.fit_transform(dft.wpt_name)
# #dft.iloc[:,['wpt_name']] = tfidf_vectorizer.get_feature_names()
# tfidf_vectorizer.vocabulary_

In [25]:
dft.wpt_name.unique()

array(['Dinamu Secondary School', 'Kimnyak', 'Puma Secondary', ...,
       'Kwambwezi', 'Bonde La Mkondoa', 'Kwa Mzee Chagala'], dtype=object)

In [26]:
dft.region_district.unique()

array(['Manyara_3', 'Arusha_2', 'Singida_2', 'Lindi_43', 'Ruvuma_3',
       'Iringa_7', 'Mtwara_4', 'Mtwara_33', 'Kilimanjaro_7',
       'Kilimanjaro_5', 'Manyara_4', 'Tabora_2', 'Mwanza_4', 'Ruvuma_2',
       'Pwani_6', 'Tanga_1', 'Dar es Salaam_3', 'Iringa_4', 'Kigoma_3',
       'Shinyanga_2', 'Dodoma_5', 'Shinyanga_1', 'Manyara_5', 'Arusha_30',
       'Tabora_1', 'Tabora_3', 'Tanga_8', 'Kilimanjaro_2', 'Morogoro_6',
       'Mbeya_2', 'Mtwara_1', 'Tanga_3', 'Shinyanga_3', 'Kilimanjaro_3',
       'Mbeya_4', 'Ruvuma_1', 'Shinyanga_8', 'Lindi_13', 'Mwanza_6',
       'Mwanza_1', 'Kilimanjaro_4', 'Ruvuma_5', 'Kagera_30', 'Mara_6',
       'Iringa_2', 'Morogoro_4', 'Kilimanjaro_1', 'Kigoma_2', 'Mbeya_6',
       'Manyara_2', 'Rukwa_2', 'Dodoma_6', 'Tabora_4', 'Kagera_7',
       'Kagera_1', 'Arusha_7', 'Kagera_2', 'Dodoma_1', 'Morogoro_1',
       'Pwani_2', 'Rukwa_1', 'Iringa_3', 'Kagera_3', 'Shinyanga_5',
       'Tabora_6', 'Dar es Salaam_2', 'Iringa_1', 'Rukwa_4', 'Morogoro_2',
       'Pwan

In [27]:
df_.region_district.unique()

array(['Iringa_5', 'Mara_2', 'Manyara_4', 'Mtwara_63', 'Kagera_1',
       'Tanga_8', 'Shinyanga_3', 'Tabora_6', 'Pwani_43', 'Ruvuma_5',
       'Shinyanga_2', 'Kilimanjaro_7', 'Shinyanga_6', 'Rukwa_2',
       'Iringa_4', 'Mwanza_1', 'Shinyanga_1', 'Kilimanjaro_3', 'Kigoma_3',
       'Kilimanjaro_4', 'Lindi_23', 'Kilimanjaro_1', 'Dodoma_4',
       'Pwani_1', 'Pwani_63', 'Arusha_2', 'Mbeya_3', 'Dodoma_1',
       'Iringa_7', 'Kigoma_1', 'Iringa_3', 'Singida_2', 'Mtwara_33',
       'Mbeya_4', 'Kilimanjaro_6', 'Mwanza_6', 'Manyara_3', 'Kagera_2',
       'Tanga_3', 'Tanga_1', 'Arusha_7', 'Singida_1', 'Kigoma_2',
       'Morogoro_3', 'Mbeya_7', 'Kagera_6', 'Tanga_2', 'Shinyanga_5',
       'Morogoro_2', 'Shinyanga_8', 'Mara_3', 'Mwanza_5', 'Iringa_1',
       'Kagera_3', 'Dodoma_5', 'Lindi_53', 'Manyara_2', 'Kagera_7',
       'Mwanza_7', 'Ruvuma_2', 'Tanga_4', 'Ruvuma_1', 'Kilimanjaro_5',
       'Kilimanjaro_2', 'Kagera_8', 'Kagera_4', 'Mbeya_2', 'Morogoro_6',
       'Mara_4', 'Manyara_5', 'Arus

In [28]:
df_.waterpoint_type.unique()

array(['communal standpipe', 'communal standpipe multiple', 'hand pump',
       'other', 'improved spring', 'cattle trough', 'dam'], dtype=object)

In [29]:
dft.waterpoint_type.unique()

array(['other', 'communal standpipe', 'hand pump',
       'communal standpipe multiple', 'improved spring', 'cattle trough',
       'dam'], dtype=object)

In [30]:
dft.shape, df_.shape

((14358, 37), (59400, 37))

In [31]:
# pd.set_option('display.max_columns', None)
#train__ = copy.deepcopy(train_input_columns)
# test_input_columns = df_.columns
test__ = list(dft.columns)
for c in train_input_columns:
    if c not in test_input_columns:
        print(f'{c} not in test')
    else:
        test__.remove(c)
print(test__)
        
    

[]


In [32]:
mc = 0
nmc = 0
for o in oc:
    tru = df_[o].unique()
    teu = dft[o].unique()
    for v in teu:
        if v in tru:
            mc+=1
        else:
            nmc+=1
mc, nmc

(14028, 10988)

In [33]:
mc = 0
nmc = 0
tru = df_['funder'].unique()
for v in dft.funder.values:
    if v in tru:
        mc+=1
    else:
        nmc+=1
mc, nmc

(13325, 1033)

In [34]:
len(df_['funder'].unique()), len(dft['funder'].unique())

(1894, 957)

In [35]:
tfidf = TfidfVectorizer(strip_accents="unicode",
                        lowercase=True,
                       )
#                 ('vect', tfidf),
encoders = Pipeline([
#                 ('tdidf', tfidf(cols=oc)),
                ('binary', BinaryEncoder(cols=binary)),
                ('onehot', ce.OneHotEncoder(use_cat_names=True,cols=hot))
            ])
df_l = df_.shape[0]
both = pd.concat([df_,dft])
print(df_l)
both.head()


59400


,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,timestamp,region_district
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1.300061,Iringa_5
8776,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.362528,Mara_2
34310,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1.361750,Manyara_4
67743,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1.359331,Mtwara_63
19728,0.0,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.310515,Kagera_1


In [36]:
both.shape

(73758, 37)

In [37]:
both_  = encoders.fit_transform(both)

In [38]:
df__  = both_.iloc[0:df_l]
dft_ = both_.iloc[df_l:]
df_l, both_.shape, df__.shape, dft_.shape

(59400, (73758, 404), (59400, 404), (14358, 404))

In [39]:
de = df__.describe()
de_test = dft_.describe()
ex_oc = df__.columns
drop = []
for o in ex_oc:
    if o in de.columns:
        if de[o]['mean'] == 0 and de_test[o]['mean'] == 0:
            drop.append(o)
        # print(de[o].mean)
X  = df__.drop(drop, axis=1)
dfp = dft_.drop(drop, axis=1)
print(drop)

['basin_-1', 'lga_-1', 'public_meeting_-1', 'recorded_by_-1', 'scheme_management_-1', 'permit_-1', 'extraction_type_-1', 'extraction_type_group_-1', 'extraction_type_class_-1', 'management_-1', 'management_group_-1', 'payment_-1', 'payment_type_-1', 'water_quality_-1', 'quality_group_-1', 'quantity_-1', 'quantity_group_-1', 'source_-1', 'source_type_-1', 'source_class_-1', 'waterpoint_type_-1', 'waterpoint_type_group_-1', 'funder_0', 'installer_0', 'wpt_name_0', 'subvillage_0', 'ward_0', 'scheme_name_0', 'region_district_0']


In [40]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    '''
    pred_exp = np.exp(preds)
    labels_exp = np.exp(labels)
    return 'error', np.sum(np.abs(pred_exp-labels_exp))/np.sum(labels_exp)
    '''
    return 'error', np.sum(np.abs(preds-labels))/np.sum(labels)

In [41]:
# read in data

df_y = pd.DataFrame(y)
print('functional needs repair', df_y[df_y.status_group == 'functional needs repair'].shape[0])
print('functional', df_y[df_y.status_group == 'functional'].shape[0])
print('non functional', df_y[df_y.status_group == 'non functional'].shape[0])
y_true = [0 if s == 'functional needs repair' else (1 if s == 'functional' else 2) 
                              for s in df_y[dc].values]

dtrain = xgb.DMatrix(X, label=y_true)
print('num_col',dtrain.num_col(), 'X', X.shape)


dtrain.set_weight([2.13 if s == 'functional needs repair' else (0.942 if s == 'functional' else 1.337) 
                              for s in df_y[dc].values])

param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
param['eta'] = 0.1
param['max_depth'] = 25
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 3
param['lambda'] = 1
param['alpha'] = 1
num_round = 50
test = xgb.DMatrix(dfp)
# watchlist = [(dtrain, 'train'), (test, 'test')]
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(test)




functional needs repair 4317
functional 32259
non functional 22824
num_col 375 X (59400, 375)


In [ ]:
dfp.shape

In [ ]:
vpreds = bst.predict(xgb.DMatrix(X))
vdf = pd.DataFrame(vpreds, columns=['v'])
print('functional needs repair',vdf[vdf.v == 0.].shape[0])
print('functional',vdf[vdf.v == 1.0].shape[0])
print('non functional', vdf[vdf.v == 2.0].shape[0])
print('r2', r2_score(y_true, vpreds))
print('accuracy', accuracy_score(y_true, vpreds))

In [ ]:
print(preds[0:50])
print(type(preds))

In [ ]:
with open('testxgb.csv', 'w') as f:
    f.write('id,status_group\n')
    for f_,i in zip(preds, dfp.index):
        index = int(f_)
        d = 'non functional' if index == 2 else ('functional' if index == 1 else 'functional needs repair')
        f.write(f"{i},{d}")
        f.write('\n')

In [ ]:
# df[dc].iloc[0:5][8776]

In [ ]:
independant_columns  = dfum.select_dtypes(exclude=['object']).columns
one = 'functional needs repair'
two = 'functional'
three = 'non functional'
for c in independant_columns:
    fig, axs = plt.subplots(ncols=3)
    fig.set_size_inches(15, 3)
    axs[0].hist(dfum[dfum[dc] == one][c])
    axs[1].hist(dfum[dfum[dc] == two][c])
    axs[2].hist(dfum[dfum[dc] == three][c])
    axs[0].set_xlabel(f'{c} for functional needs repair')
    axs[1].set_xlabel(f"{c} for functional")
    axs[2].set_xlabel(f"{c} for non functional")
    plt.show()

In [ ]:
# !kaggle competitions download -c ds1-predictive-modeling-challenge

In [ ]:
17109 + 38845, 14031 + 10989 , 2 * (14031 + 10989)
